In [2]:
import torch
import torch.nn as nn
from torchvision.models import resnet50

In [ ]:
class UpsampleBlock(nn.Module):
    def __init__(self, in_channel, out_channel, mode='nearest'):
        super(UpsampleBlock, self).__init__()
        self.unsample = nn.Sequential([
            nn.Upsample(scale_factor=2, mode=mode),
            nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm2d(out_channels)
            nn.ReLU(inplace=True)
        ])

    def forward(self, x):
        return self.unsample(x)